## Loading Libraries

In [1]:
import tensorflow
import numpy as np
import json
import requests
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

Using TensorFlow backend.


## Loading quantized client model

In [3]:
interpreter = tensorflow.lite.Interpreter(model_path="VGG16_Client.tflite")
#allocate the tensors
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

## REST endpoint

In [14]:
endpoint = 'http://10.131.36.51:8501/v1/models/VGG16_Server:predict'

[[('n02090721', 'Irish_wolfhound', 0.425627798), ('n02091831', 'Saluki', 0.183746397), ('n02113799', 'standard_poodle', 0.110293746), ('n02109047', 'Great_Dane', 0.0402301066)]]


## Iterating a dir for all images .jpg & .jpeg

In [ ]:
# Gets the path of the directory where the images are located as a command line argument
folder_dir = sys.argv[1]
for images in os.listdir(folder_dir):
    # Operates on .jpg and .jpeg file types
    if images.endswith(".jpg") or images.endswith(".jpeg"):
        image = load_img(folder_dir+images, target_size=(224, 224))
        # convert the image pixels to a numpy array
        input_shape = input_details[0]['shape']
        input_tensor= np.array(np.expand_dims(image,0), dtype=np.float32)
        input_index = interpreter.get_input_details()[0]["index"]
        output_index = interpreter.get_output_details()[0]["index"]
        interpreter.set_tensor(input_index, input_tensor)

        interpreter.invoke()
        output_details = interpreter.get_output_details()
        predictions = interpreter.get_tensor(output_index)
        data = json.dumps({"instances": predictions.tolist()})
        # HTTP POST to server with HTTP endpoint and pre-processed data
        json_response = requests.post(endpoint, data)
        prediction = json.loads(json_response.text)
        result = decode_predictions(np.array(prediction['predictions']), top=4)
        print(result)